
### Librerías

In [1]:
import sys
import pandas as pd
import GetOldTweets3 as got
import sys
import matplotlib.pyplot as plt
from googletrans import Translator
import socket as sk
from datetime import *
import tweepy

### Configuración de parámetros 

In [2]:
pd.options.display.max_columns = 5000
pd.options.display.max_rows = 5000
sk.setdefaulttimeout(600)

#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=3600)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('max_colwidth',-1)

### Ejecución de clases y métodos del proyecto

In [3]:
%run Metodos_Propios.ipynb
%run Gestion_Ficheros.ipynb
%run GetOldTweets.ipynb
%run NLP.ipynb
%run Extract_Twitter_v2.ipynb

# Método Principal - TFM Data Science

In [19]:
if __name__ == '__main__':
    
    #Control horario programa
    fecha_inicio = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    print('Comienzo del programa: ',fecha_inicio)
    
    # Definición de variables
    fichero = 'CSVs/info_tweets_el_corte_ingles_202003.csv'
    pattern = 'CSVs/info_tweets_el_cor*.csv'
    fichero_user = 'CSVs/following_info_el_corte_ingles.csv'
    fichero_tweets = 'CdM/Tweets_Info.csv'
    fichero_fact = 'CdM/Tweets_Fact_Info.csv'
    following_ids = []  
    calendario = calendar()
    df_tweets, df_tweets_final, df_following, df_fact_following = creacion_DataFrame()
    positivo, negativo, neutral, sentiment = creacion_Variables()
    
    #Flujo de la información
    online = 0
    all_tweets = 0
    online_user = 0
    
    #Definición POO
    file = File_Manage()
    file.initValues(fichero, pattern)
    gotTweets = GoT(calendario)
    nlp = NLP()

    
    #Acceso a Twitter
    api = acceso_Twitter() 

    
    #Obtención por GetOldTweets
    if online == 1:
        print('Opción Online')
        df_tweets = gotTweets.get_Tweets(got.getCalendar())
    #Lectura de ficheros para ir más rápido ubicados en CSVs
    elif all_tweets == 1:
        print('Opción Offline Tweets')
        df_tweets = file.read_all_Tweets(file.getPattern()) 
    else: 
        print('Opción Un fichero: ', file.getFile())
        df_tweets = file.read_Tweets(file.getFile())
    
    df_tweets['valor'] = -10
    df_tweets['fecha'] = df_tweets['date'].astype(str).str[:10]
    df_tweets['fecha'] = df_tweets['fecha'].str.replace('-','/')
    df_tweets['idioma'] = 'NA' 
    

    #Pasando el algoritmo de NLP para obtener el análisis de sentimiento
    df_tweets_final = nlp.analisis_2(df_tweets)
    #Generación de CSVs para el CdM

    file.setFile(fichero_tweets)
    file.writeCSV(df_tweets_final,file.getFile())
    
    #Para el caso de los usuarios followings, se crea un minimodelo de datos
    if online_user == 1:
        following_ids = get_following_ids(api)
        following_limit=[following_ids[n:n+99] for n in range(0, len(following_ids), 100)]
        df_following = pd.DataFrame(get_info_following(api,following_limit))
        file.setFile(fichero_user)
        file.writeCSV(df_following,file.getFile())
    else:
        file.setFile(fichero_user)
        df_following = file.readCSV(df_following,file.getFile())
     
    df_tweets.columns = ['id_text','created_at_tweet','name_account','text','geo','valor','fecha','idioma']
    df_following.columns = ['id_account','name_account','screen_name_account','created_at_account','location_account','followers_account','friends_account','description_account']
    df_fact_following = pd.merge(df_tweets, df_following,on=['name_account'],how='left')
    
    
    
    file.setFile(fichero_fact)    
    file.writeCSV(df_fact_following,file.getFile())
    
    fecha_fin = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    print('Comienzo Finalizacion: ',fecha_fin)
    

Comienzo del programa:  08/04/2020, 20:55:11
Opción Un fichero:  CSVs/info_tweets_el_corte_ingles_202003.csv
Comienzo Finalizacion:  08/04/2020, 20:55:24


In [20]:
df_fact_following

,id_text,created_at_tweet,name_account,text,geo,valor,fecha,idioma,id_account,screen_name_account,created_at_account,location_account,followers_account,friends_account,description_account
0,1242155509186211841,2020-03-23 18:25:26+00:00,CarlosOnRetail,"In Spain, @elcorteingles is prioritising grocery orders for medical personnel via the set up of a dedicated telephone number/email address for a quicker and more convenient shopping. El Corte Inglés is also offering them free delivery.",NaN,1,2020/03/23,NA,2183187386,CarlosOnRetail,2013-11-08 23:24:37,Madrid,1175,653,"Retail Analyst analysing the Spanish, Portuguese and Latin American markets."
1,1237806657855504390,2020-03-11 18:24:39+00:00,CarlosOnRetail,"Despite #COVID19 worrying news, it is business as usual at most Madrid food stores such as this El Corte Inglés food hall in Central Madrid.",NaN,1,2020/03/11,NA,2183187386,CarlosOnRetail,2013-11-08 23:24:37,Madrid,1175,653,"Retail Analyst analysing the Spanish, Portuguese and Latin American markets."
2,1240294803050573824,2020-03-18 15:11:39+00:00,xataka,"Comprar por Internet en tiempos de Coronavirus: éstos son los planes de reparto de Amazon, Mercadona, El Corte Inglés y otras tiendas https://www.xataka.com/empresas-y-economia/comprar-internet-tiempos-coronavirus-estos-planes-reparto-principales-ecommerce-espana",NaN,0,2020/03/18,NA,18200400,xataka,2008-12-17 21:28:48,España y Latinoamérica,1407169,41,Xataka es la publicación líder de tecnología en español. Apasionados por la tecnología. En YT: https://t.co/sp4Jos5d4V y en IG: https://t.co/bLhSiCQiro
3,1238524881525977088,2020-03-13 17:58:37+00:00,RetailerSpain,El Corte Inglés activa un plan de crisis ante el desplome de las ventas por el Covid-19,NaN,0,2020/03/13,NA,2614262096,RetailerSpain,2014-07-09 21:38:20,NaN,2543,1638,¿Nos sigues en la aventura del #Retail ?#WeLoveRetail #RetailActivo #LoSabemosAntesQueNadie
4,1238524825464954880,2020-03-13 17:58:23+00:00,RetailerSpain,El Corte Inglés cierra sus centros y manda a casa a la mayoría de sus 80.000 empleados,NaN,0,2020/03/13,NA,2614262096,RetailerSpain,2014-07-09 21:38:20,NaN,2543,1638,¿Nos sigues en la aventura del #Retail ?#WeLoveRetail #RetailActivo #LoSabemosAntesQueNadie
5,1234792709874999296,2020-03-03 10:48:18+00:00,RetailerSpain,Renatta&Go traza una nueva hoja de ruta: da un paso atrás en El Corte Inglés para centrarse en retail https://www.modaes.es/empresa/renattago-traza-una-nueva-hoja-de-ruta-da-un-paso-atras-en-el-corte-ingles-para-centrarse-en-retail.html vía @modaes.es,NaN,0,2020/03/03,NA,2614262096,RetailerSpain,2014-07-09 21:38:20,NaN,2543,1638,¿Nos sigues en la aventura del #Retail ?#WeLoveRetail #RetailActivo #LoSabemosAntesQueNadie
6,1237716742409007104,2020-03-11 12:27:21+00:00,chicadelatele,El supermercado de El Corte Inglés también,NaN,0,2020/03/11,NA,4473901,chicadelatele,2007-04-13 08:53:35,NaN,6863,184,"No soy periodista y solo hago opinión, personal y absolutamente subjetiva"
